In [ ]:
%matplotlib inline

In [ ]:
import os
import numpy as np
from PIL import Image
from dotenv import load_dotenv
from matplotlib import pyplot as plt

from utils.predictors import ImageClassificationPredictor

In [ ]:
load_dotenv("env")

In [ ]:
model_predictor = ImageClassificationPredictor(
    model_id=os.environ["IMAGE_CLASSIFICATION_MODEL_ID"],
    model_version=os.environ["IMAGE_CLASSIFICATION_MODEL_VERSION"],
    model_uri=os.environ["IMAGE_CLASSIFICATION_MODEL_URI"],
    instance_type=os.environ["INFERENCE_INSTANCE_TYPE"]
)

In [ ]:
%%time
model_predictor.deploy(
    instance_type=os.environ["INFERENCE_INSTANCE_TYPE"],
    instance_count=int(os.environ["INFERENCE_INSTANCE_COUNT"]),
)

In [ ]:
def sort_label_by_probability(labels, probabilities):
    sorted_idx = sorted(
        range(len(probabilities)), key=lambda index: probabilities[index], reverse=True
    )
    return [(labels[idx], probabilities[idx]) for idx in sorted_idx]


image_dir = os.path.join("images", "test")
for image_filename in sorted(os.listdir(image_dir)):
    if not image_filename.endswith(".jpg"):
        continue
    image_path = os.path.join(image_dir, image_filename)
    with open(image_path, "rb") as file:
        image_binary = file.read()
    labels, probabilities = model_predictor.predict(image_binary)
    image = Image.open(image_path)
    plt.figure(figsize=(5, 5))
    plt.imshow(np.array(image))
    plt.show()
    print(image_path)
    for label, probability in sort_label_by_probability(labels, probabilities):
        print(f"{label}: {probability*100:.2f}%")
    print("")

In [ ]:
# Delete the SageMaker endpoint and the attached resources
model_predictor.delete()